In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image

In [2]:
#彩色图均衡化看看有什么变化

def ImageHe(img):

    clahe=cv2.createCLAHE(clipLimit=2.0,tileGridSize=(8,8))

    r,b,g=cv2.split(img)
    rh=clahe.apply(r)
    bh=clahe.apply(b)
    gh=clahe.apply(g)

    img_new=cv2.merge((rh,bh,gh))
    return img_new

In [3]:
#边缘检测

def EdgeDetect(img):
    gray=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),0)
    edges=cv2.Canny(blur,100,200)
    return edges

In [4]:
# 选取兴趣区域

def RegionOfIn(img):
    w,h=img.shape[0],img.shape[1]
    mask=np.zeros_like(img)
    if len(img.shape)>2: 
        mask_color=(255,)*img.shape[2]
    else:
        mask_color=(255,)
    
    vertices=np.array([[(0,w),(580,300),(600,300),(h,w)]])
    
    cv2.fillPoly(mask,vertices,mask_color)

    masked_image = cv2.bitwise_and(img, mask)
    return masked_image



In [5]:
# 绘制直线
def findpoint(lines,ymax):
    x=[p[0][0] for p in lines]
    y=[p[0][1] for p in lines]
    a,b = np.polyfit(y,x,1)
    xmax=int(a*ymax+b)
    return xmax,a,b

    
def hough_lines(img, min_line_len,max_line_gap):
    lines = cv2.HoughLinesP(img, 1, np.pi/180, 20, minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    right_lines=[]
    left_lines=[]
    for line in lines:
        for x1,y1,x2,y2 in line:
            if x1!=x2:
                k=(y2-y1)/(x2-x1)
                if k>0.3:
                    right_lines.append([(x1,y1)])
                    right_lines.append([(x2,y2)])
                elif k<0:
                    left_lines.append([(x1,y1)])
                    left_lines.append([(x2,y2)])
    if (len(left_lines) <= 0 or len(right_lines) <= 0):
        return line_img
    ymax=img.shape[0]
    right_max,a_1,b_1=findpoint(right_lines,ymax)
    left_max,a_2,b_2=findpoint(left_lines,ymax)
    midpoint_y=int((b_2-b_1)/(a_1-a_2))
    midpoint_x=int(a_1*midpoint_y+b_1)
    cv2.line(line_img,(midpoint_x,midpoint_y),(right_max,ymax),(255,0,0),10)
    cv2.line(line_img,(midpoint_x,midpoint_y),(left_max,ymax),(255,0,0),10)
    
    return line_img

In [6]:
#融合图像

def weighted_img(img1,img2, a=0.8,b=1.):
    return cv2.addWeighted(img2,a,img1,b,0)

In [7]:
def process_img(img):
    img_new=ImageHe(img)
    edge_img=EdgeDetect(img_new)
    edge_cut_img=RegionOfIn(edge_img)
    line_img=hough_lines(edge_cut_img,20,20)
    final_img=weighted_img(line_img,img)
    
    return final_img

In [ ]:
cap = cv2.VideoCapture('solidWhiteRight.mp4')

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        frame=process_img(frame)

        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
img= np.array(Image.open('sample_color.jpg'))
plt.imshow(img)

In [ ]:
plt.imshow(process_img(img))